In [17]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import re
import os
from openpyxl import load_workbook
from shutil import move
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait

In [18]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument('headless')
# options.add_argument('Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like Mac OS X) AppleWebKit/604.3.5 (KHTML, like Gecko) Version/13.0 MQQBrowser/10.1.1 Mobile/15B87 Safari/604.1 QBWebViewUA/2 QBWebViewType/1 WKType/1')
chrome = webdriver.Chrome('./chromedriver')
data = []
tStart = time.time()
for i in range(1,35):
    chrome.get(url.format(i,i))
    chrome.find_element_by_id('twotabsearchtextbox').send_keys(Keys.F12)
    print(url.format(i,i))
#     time.sleep(5)
    soup = BeautifulSoup(chrome.page_source,'lxml')
    amazon_card=soup.find_all(class_="a-size-medium a-color-base a-text-normal")
    if len(amazon_card) == 0:
        print("找不到顯示卡標題")
        break
    data.append([ele.text for ele in amazon_card])   
chrome.close()    
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=1&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_1
https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=2&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_2
https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=3&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_3
https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=4&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_4
https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=5&language=zh_TW&qid=1598425548&rnid=179236710

In [19]:
amazon_json = json.load(open("AMAZON_COMPUTER_PRICE" + ".json",encoding="utf-8"))
print(amazon_json['title'])

['MSI微星', 'NVIDIA', 'ZOTAC', 'GIGABYTE', 'EVGA', 'PowerColor', 'XFX', 'MSI', 'Gigabyte', 'Sapphire']


In [20]:
title=[]
for adx,datas in enumerate(data):
    df = pd.DataFrame(datas,columns=['f'])
    columns_of_datas = [ddd for ddd in range(0,len(datas))]  
    # 分種類並寫入json中
    for aaa in columns_of_datas:
    # 正則表達式將標題的牌子切割出來
        matchObj = re.split(r'\s', df['f'][aaa], re.M|re.I)
        title.append(matchObj[0])
# 篩選重複的牌子        
json_title_dumps = list(set(title))
json_title = {'title': json_title_dumps}
amazon_json.update(json_title)
# 將整理好title的json寫入json檔案中
with open('AMAZON_COMPUTER_PRICE.json', 'w', encoding='utf-8') as f:
    json.dump(amazon_json, f, ensure_ascii=False, indent=4,sort_keys=True)
print(amazon_json)

{'sheet_name': '顯示卡', 'title': ['Generic', 'ASROCK', 'MHQJRH', 'MOTZU', 'Alienware', 'I/O', 'LED', 'Dual', 'Crossfire', 'Savant', '華碩游戏图案卡', 'upHere', 'OEM', 'PNY', 'MSI微星', 'ATI', 'COMeap', 'COOCAT', 'Wendry', 'EVGA', '256MB', 'Geforce', 'Aiposen', 'K620', 'PCIe', 'BestParts', 'XtremeAmazing', 'Parent-GL05', 'CloverTale', 'Nvidia', 'PLA09215B12H', 'Sapphire', 'Yeston', 'Dummy', 'HP', 'Graphics', 'ASHATA', 'AMD', 'SLI', 'MSI', '四級製造，阿特拉斯顯卡支撐。', 'Guts', 'Arm', 'Funtin', 'NVIDIA', '藍寶石', 'coolxan', 'inRobert', 'GATEWAY', 'VDWall', 'ASRock', 'Linsn', 'Gigabyte', 'GIGABYTE', 'Jonsbo', 'CERRXIAN', 'upHere-GS05BK/GS05CF/GS05ARGB/GL05', 'New', 'StarTech.com', '鋁製', 'Zopsc', '微星', '技嘉', 'Epic', 'ADT-LINK', 'PCIE', '0X31G', 'Inno3D', 'ASUS', 'Cooler', 'MB', 'M2', 'QNAP', '[Carbon', '100mm', 'ViewMax', 'Supermicro', 'GLOTRENDS', 'Aquamoon', 'Xixun', 'ZOTAC', 'Smart', 'PowerColor', 'Zotac', 'Biostar', 'Bewinner1', '3DFX', 'Asus', 'Brace', 'nVIDIA', 'Bitspower', '661-5968', 'Diamond', 'The', 'Lapt

In [10]:
cwd = os.getcwd()
curr_file = cwd + "\\" + output_file
target_file = target_path + output_file

try:
    move(curr_file, target_file)
    print(output_file + "\n移動檔案完畢.")
except Exception as e:
    print(output_file + "\n移動檔案失敗.")
    print(e.args)

20200925_amazon_graphics card.xlsx
移動檔案完畢.


# 建立excel跟洗滌資料並且存入excel

In [24]:
now_date = datetime.strftime(datetime.now() , '%Y%m%d')
output_file = now_date + "_amazon_graphics card.xlsx"
target_path = "D:\\Amazon_graphics_card\\"
writer = pd.ExcelWriter(output_file, engine='openpyxl')
# 20200924  
title_dict={}
def title_var(title,datas):
    for s_datas in datas:
        for s_title in title:
            if s_datas.split(s_title+' ')[0] == '': 
                title_dict.setdefault(s_title,[]).append(s_datas)
    return title_dict


for adx,datas in enumerate(data):   
    title_dict = title_var(amazon_json['title'],datas)   

df = pd.DataFrame.from_dict(title_dict, orient='index').T
df.to_excel(writer,sheet_name=amazon_json['sheet_name'],startrow=0,startcol=0,index=False) 
writer.save() 

                                                  XFX  \
0                              XFX Thicc Boost Up 顯示卡   
1                                         XFX HDMI 顯卡   
2   XFX RX 5700 XT 三重耗 8GB GDDR6 3xDP HDMI 顯示卡 RX-...   
3                                         XFX HDMI 顯卡   
4                              XFX Thicc Boost Up 顯示卡   
..                                                ...   
70                                               None   
71                                               None   
72                                               None   
73                                               None   
74                                               None   

                                                MSI微星  \
0   MSI微星 Gaming GeForce GTX 1660 192 位元 HDMI/DP 6...   
1   MSI微星 Gaming GeForce RTX 2060 6GB GDRR6 192 位元...   
2   MSI微星 Gaming GeForce GTX 1660 Super 192 位元 HDM...   
3   MSI微星 Gaming GeForce GT 710 2GB GDRR3 64 位元 HD...   
4   MSI微星 Gaming Radeon RX 560